给定一个问题和解决方案，先扩写它们，再生成专利背景介绍、权要和实施方式。

In [1]:
import os
from dotenv import load_dotenv
from datetime import datetime

import openai
from IPython.display import HTML, display
from ipywidgets import widgets
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma

load_dotenv()

True

In [2]:
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
print(openai.api_type, openai.api_base, openai.api_version)

llm = ChatOpenAI(engine='test0406', model_name="gpt-3.5-turbo", temperature=0.7, max_tokens=4096)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


azure https://test0406.openai.azure.com/ 2023-03-15-preview


In [3]:
template_rewriteq = '''你是一个专业的流程挖掘领域的专利代理，你的任务是帮助专利发明人完善发明点设计。请根据发明人提供的简要描述，重新编写专利发明点要解决问题的说明。
问题说明需要包含以下几个问题涉及的内容：
1，待解决的问题属于哪个技术领域？
2，这个问题的一般解决方法是什么？存在什么样的缺陷或者弊端？
3，为什么要解决这个问题？有多大的现实价值？
4，这个问题的完整描述（包含业务场景、技术背景）是什么？
如果原来的描述中不包含上述问题的答案，请补充相关的内容。

待解决问题的简单描述: {raw_question}

根据上述简单描述，重新生成的更为完整、详细的问题说明：'''

prompt_rewriteq = PromptTemplate(input_variables=['raw_question'], template=template_rewriteq)
rewriteq_chain = LLMChain(llm=llm, prompt=prompt_rewriteq, output_key='question')

In [4]:
template_rewritea = '''你是一个专业的流程挖掘领域的专利代理，你的任务是帮助专利发明人完善发明点设计。请根据待解决问题的描述，以及简单的解决方案描述，重新描述专利发明点的解决方案。
解决方案要包含以下几个问题：
1，解决方案的基本思路是什么？
2，它与一般的解决方法有什么不同？这些不同点可以产生什么样的价值？
3，这个解决方案是通用的，还是特定于具体场景的？

待解决问题的描述: {question}
解决方案的简单描述: {raw_solution}

请根据上述信息，重新生成更为完整、详细的解决方案描述：'''

prompt_rewritea = PromptTemplate(input_variables=['question', 'raw_solution'], template=template_rewritea)
rewritea_chain = LLMChain(llm=llm, prompt=prompt_rewritea, output_key='solution')

In [5]:
template_background = '''你是一个专业的流程挖掘领域的专利代理，帮助专利发明人设计发明点设计，并起草专利草稿。请根据以下待解决问题及解决方案的描述，起草专利的背景介绍部分。
专利的背景介绍要包含所解决问题所在的技术领域介绍，问题的简单介绍，以及这个问题所产生的不良影响。
待解决问题的描述: {question}
解决方案的描述: {solution}
请生成这个专利发明点的背景介绍：'''

prompt_background = PromptTemplate(input_variables=['question', 'solution'], template=template_background)
background_chain = LLMChain(llm=llm, prompt=prompt_background, output_key='background')

In [6]:
template_claims = '''你是一个专业的流程挖掘和任务挖掘方面的专利代理，负责专利发明点设计和专利文档起草。请根据以下要解决的问题及解决方案，起草专利的权利要求部分。
在设计专利的权利要求时要一步步来，先判断解决方案的创新性和价值所在，然后再根据创新点来设计对应的权利要求。针对要解决的问题，先将解决方案重新描述为多个步骤，再判断每一个步骤
是否有必要存在，若是没有必要，则可以删除；对于必要存在的步骤，再考虑它是否具备创新性。若是不具备，则更新对应的步骤。只要其中一个步骤具备创新性即可。
可使用以下问题来引导生成过程：

要解决的问题: {question}
提供的解决方案: {solution}
请生成这个专利发明点的权利要求：'''

prompt_claims = PromptTemplate(input_variables=['question', 'solution'], template=template_claims)
claims_chain = LLMChain(llm=llm, prompt=prompt_claims, output_key='claims')

In [7]:
template_explanation = '''你是一个专业的流程挖掘和任务挖掘方面的专利代理，负责专利发明点设计和专利文档起草。请根据解决的问题、解决方案及权利要求，
扩充权利要求部分的内容。扩充的方法是，先设计一个具体的应用场景，将权利要求转换为具体的实现步骤，并结合应用场景进行描述每一步的输入输出，以及中间的处理过程。
若是不同权利要求对应的操作步骤是相同的，则只需要在第一次给出描述，后面的则省略。
可使用这些问题来引导生成过程：它的主要算法或者技术是什么？需要什么样的软硬件运行环境？是否有多种实现方式？每种实现方式最简单的形式是什么样的？

要解决的问题: {question}
提供的解决方案: {solution}
独立权利要求: {claims}

请将权利要求扩展为具体的实现步骤，并给出相应的示例说明：'''

prompt_explanation = PromptTemplate(input_variables=['question', 'solution', 'claims'], template=template_explanation)
explanation_chain = LLMChain(llm=llm, prompt=prompt_explanation, output_key='explanation')

In [8]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[rewriteq_chain, rewritea_chain, background_chain, claims_chain, explanation_chain],
    input_variables=["raw_question", "raw_solution"],
    # Here we return multiple variables
    output_variables=["question", "solution", "background", "claims", "explanation"],
    verbose=True)

In [9]:
question = '''为了还原用户UI操作流程，需要识别哪些UI操作是相同的，哪些是不同的。比如，提交报销单，两个人填写的报销单可能只有姓名、部门、时间这些内容是相似的，
其他的比如出差事由、时间范围等等相差非常大，在页面上看起来也很不一样。但是，它们同属于一个业务操作，应该判断相同操作，即使对应的窗口/应用从视觉上和内容上来看相差非常大。
仅用图片相似性无法解决这个问题，因为文字内容变化难以检测；仅用文字比较也不行，因为内容相差很大的UI操作可能属于同一个业务操作。
'''

solution = '''使用CV算法识别窗口/应用的区域，同一个窗口的区域分布应该是完全一样的。另外，再提取一两个关键标签，代表当前操作的内容。只要窗口区域相似，
关键标签相同，就可以判断两个UI操作属于同一个业务操作。识别窗口区域的方法是canny，通过边框识别窗口的主要布局，同一个应用内只有有限的窗口，每个窗口的布局是很少变化的。
提交关键标签，是指使用OCR识别窗口中的所有文字，然后提取关键位置的文字，比如窗口标题栏中的文字。两者结合就可以较有效地区分不同窗口，进而判断窗口的相似性。
'''

result = overall_chain({'raw_question': question, 'raw_solution': solution})



> Entering new SequentialChain chain...

> Finished chain.


In [10]:
print(result['question'])

1. 待解决的问题属于UI操作流程挖掘领域。
2. 一般解决方法是通过比较UI操作的图像和文字内容来识别相同的业务操作。但是这种方法存在文字内容变化难以检测的缺陷，以及内容相差很大的UI操作可能属于同一个业务操作的弊端。
3. 解决这个问题可以提高用户操作流程的还原度，从而提高用户体验和减少误操作，具有重要的现实价值。
4. 该问题是在UI操作流程挖掘领域中，为了还原用户的操作流程，需要识别哪些UI操作是相同的，哪些是不同的。比如，在提交报销单的业务操作中，两个人填写的报销单可能只有姓名、部门、时间这些内容是相似的，其他的比如出差事由、时间范围等等相差非常大，在页面上看起来也很不一样。但是，它们同属于一个业务操作，应该判断相同操作，即使对应的窗口/应用从视觉上和内容上来看相差非常大。目前，仅用图片相似性无法解决这个问题，因为文字内容变化难以检测；仅用文字比较也不行，因为内容相差很大的UI操作可能属于同一个业务操作。因此，需要寻找一种新的方法来识别相同的业务操作。


In [11]:
print(result['solution'])

解决方案的基本思路是使用计算机视觉（CV）算法来识别窗口/应用的区域，并提取关键标签，以便判断两个UI操作是否属于同一个业务操作。具体实现方法包括以下步骤：

1. 使用Canny算法识别窗口的边框，确定窗口的主要布局。由于同一个应用内只有有限的窗口，每个窗口的布局是很少变化的，因此可以通过比较窗口布局的相似程度来判断两个窗口是否属于同一个应用。

2. 使用OCR技术识别窗口中的所有文字，并提取关键位置的文字，比如窗口标题栏中的文字。这些关键标签可以代表当前操作的内容，有助于更准确地判断两个UI操作是否属于同一个业务操作。

3. 将窗口区域的相似性和关键标签的相同性结合起来，判断两个UI操作是否属于同一个业务操作。只要窗口区域相似，关键标签相同，就可以认为这两个UI操作是相同的业务操作。

与一般的解决方法相比，这个解决方案不仅可以处理文字内容变化难以检测的情况，还可以解决内容相差很大的UI操作可能属于同一个业务操作的问题。同时，这个解决方案可以提高用户操作流程的还原度，从而提高用户体验和减少误操作，具有重要的现实价值。

这个解决方案是通用的，可以应用于各种不同的UI操作流程挖掘场景。无论是提交报销单、填写表单、还是其他业务操作，都可以使用这个解决方案来判断UI操作是否属于同一个业务操作。


In [12]:
print(result['background'])

本发明点涉及UI操作流程挖掘领域，特别是一种识别相同业务操作的方法。当前，现有技术通过比较UI操作的图像和文字内容来识别相同的业务操作，但存在文字内容变化难以检测的缺陷，以及内容相差很大的UI操作可能属于同一个业务操作的弊端。针对这些问题，本发明点提出了一种基于计算机视觉算法的解决方案，通过识别窗口区域和提取关键标签来判断UI操作是否属于同一业务操作，从而提高用户操作流程的还原度，进而提高用户体验和减少误操作。与现有技术相比，本发明点的解决方案可以处理文字内容变化难以检测的情况，同时可以解决内容相差很大的UI操作可能属于同一个业务操作的问题，具有重要的现实价值。该解决方案通用性强，适用于各种不同的UI操作流程挖掘场景，例如提交报销单、填写表单等。


In [13]:
print(result['claims'])

1. 一种用于识别UI操作流程中相同业务操作的方法，包括以下步骤：
    a. 使用Canny算法识别窗口边框，确定窗口的主要布局；
    b. 使用OCR技术识别窗口中的关键标签；
    c. 将窗口区域的相似性和关键标签的相同性结合起来，判断两个UI操作是否属于同一个业务操作。
2. 根据权利要求1中的方法，进一步包括：
    a. 在步骤b中，识别窗口标题栏中的文字作为关键标签；
    b. 在步骤c中，只要窗口区域相似，关键标签相同，就可以认为这两个UI操作是相同的业务操作。
3. 根据权利要求1或2中的方法，应用于提交报销单的业务操作，识别相同的业务操作。
4. 根据权利要求1或2中的方法，应用于填写表单的业务操作，识别相同的业务操作。
5. 根据权利要求1或2中的方法，应用于任何UI操作流程挖掘场景中，识别相同的业务操作。


In [14]:
print(result['explanation'])

权利要求1：一种用于识别UI操作流程中相同业务操作的方法，包括以下步骤：
a. 使用Canny算法识别窗口边框，确定窗口的主要布局；
b. 使用OCR技术识别窗口中的关键标签；
c. 将窗口区域的相似性和关键标签的相同性结合起来，判断两个UI操作是否属于同一个业务操作。

实现步骤：

步骤a：使用Canny算法识别窗口边框，确定窗口的主要布局
1. 输入：窗口截图
2. 输出：窗口边框
3. 中间处理：使用Canny算法提取边缘特征，使用霍夫变换检测直线，确定窗口的边框
4. 示例说明：对于一个报销单的窗口截图，使用Canny算法提取边缘特征，检测出窗口的边框如图1所示。

步骤b：使用OCR技术识别窗口中的关键标签
1. 输入：窗口截图
2. 输出：窗口中的关键标签
3. 中间处理：使用OCR技术识别窗口中的所有文字，通过比较位置、字体、字号等信息，确定窗口中的关键标签
4. 示例说明：对于一个报销单的窗口截图，使用OCR技术识别窗口中的所有文字，并确定窗口标题栏中的文字为关键标签，如图2所示。

步骤c：将窗口区域的相似性和关键标签的相同性结合起来，判断两个UI操作是否属于同一个业务操作
1. 输入：两个窗口截图
2. 输出：判断结果，是否属于同一个业务操作
3. 中间处理：比较两个窗口的边框相似度，比较两个窗口中的关键标签是否相同，综合判断是否属于同一个业务操作
4. 示例说明：对于两个窗口截图，比较它们的边框相似度和关键标签是否相同，如果相似度和相同性都达到一定阈值，则判断它们属于同一个业务操作，如图3所示。

权利要求2：根据权利要求1中的方法，进一步包括：
a. 在步骤b中，识别窗口标题栏中的文字作为关键标签；
b. 在步骤c中，只要窗口区域相似，关键标签相同，就可以认为这两个UI操作是相同的业务操作。

实现步骤：

步骤a：在步骤b中，识别窗口标题栏中的文字作为关键标签
1. 输入：窗口截图
2. 输出：窗口标题栏中的文字
3. 中间处理：使用OCR技术识别窗口中的所有文字，通过比较位置、字体、字号等信息，确定窗口标题栏中的文字为关键标签
4. 示例说明：对于一个报销单的窗口截图，使用OCR技术识别窗口中的所有文字，并确定窗口标题栏中的文字为关键标签，如图2所示。

步骤b：在步骤c中，只要窗口区域相似，关键标签相同，就可以认为这两个UI操作是相同

In [15]:
with open('patents/patent-rpm-test1.txt', 'w') as f:
    f.write('question: ' + result['raw_question'])    
    f.write('\n\nsolution: ' + result['raw_solution'])
    f.write('\n\nbackground: ' + result['background'])
    f.write('\n\nclaims: ' + result['claims'])
    f.write('\n\nexplanation: ' + result['explanation'])